## Advanced Lane Finding Video

In [19]:
import numpy as np
from collections import deque
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Class to keep chracteristics of line

In [20]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, n):
        # length of storage
        self.n = n
        # number of valid vaue
        self.n_valid = 0
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = deque([], maxlen=n) 
        #average x values of the fitted line over the last n iterations
        self.best_x = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #I added for averaged curvature
        self.best_curvature = None
        #I added for current coefficients
        self.fit = np.array([0,0,0], dtype='float') 
        #I added for the last n of coefficients
        self.recent_fit = deque([], maxlen=n) 
        #I added for the last n of curvature
        self.recent_curvature = deque([], maxlen=n) 
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [21]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dirs = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(dirs)
    binary_output[(dirs >= thresh[0]) & (dirs <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output
    
def color_thresh(img, s_thresh=(170, 255), sx_thresh=(20, 100), l_thresh=(40,255)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Threshold lightness
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    #combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    combined_binary[((l_binary == 1) & (s_binary == 1) | (sxbinary==1))] = 1
    return combined_binary

def threshold_binary(img):
    # color and gadient
    color_binary = color_thresh(img)
    # direction of the threshold
    dir_binary = dir_threshold(img, sobel_kernel=15, thresh=(0.7, 1.3))
    # maginitude of the gradient
    mag_binary = mag_thresh(img, sobel_kernel=3, mag_thresh=(30, 100))
    # gradient
    gradx = abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=100)
    grady = abs_sobel_thresh(img, orient='y', thresh_min=20, thresh_max=100)
    # Plot the result
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (color_binary == 1)] = 1
    
    return combined

## Apply a perspective transform to rectify binary image ("birds-eye view").
### Todo: is there better way to set source and destination rectangle?

In [22]:
def transform_perspective(img):
    img_size = (img.shape[1], img.shape[0])

    src = np.float32(
        [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
        [((img_size[0] / 6) - 10), img_size[1]],
        [(img_size[0] * 5 / 6) + 60, img_size[1]],
        [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
    dst = np.float32(
        [[(img_size[0] / 4), 0],
        [(img_size[0] / 4), img_size[1]],
        [(img_size[0] * 3 / 4), img_size[1]],
        [(img_size[0] * 3 / 4), 0]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
        
    return warped, M


## Finding The Lines by Sliding Window

In [23]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
        
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = int(leftx_current - margin)  # Update this
        win_xleft_high = int(leftx_current + margin)  # Update this
        win_xright_low = int(rightx_current - margin) # Update this
        win_xright_high = int(rightx_current + margin) # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
                
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(ploty, binary_warped):
    global lineL, lineR
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    try:
        lineL.detected = True
        lineR.detected = True
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        lineL.detected = False
        lineR.detected = False
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    lineL.current_fit = left_fit
    lineL.fit = left_fitx
    lineL.allx = leftx
    lineL.ally = lefty
    lineR.current_fit = right_fit
    lineR.fit = right_fitx
    lineR.allx = rightx
    lineR.ally = righty

    return

def fit_poly(ploty, leftx, lefty, rightx, righty):
    global lineL, lineR
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    lineL.current_fit = left_fit
    lineR.current_fit = right_fit
    return left_fitx, right_fitx

def search_around_poly(ploty, binary_warped, left_fit, right_fit):
#    global count
    global lineL, lineR
#    print('Debug: ' + str(count) + ' search_around:' + str(left_fit) + ':' + str(right_fit))
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 80

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    if len(leftx) < 2 or len(rightx) < 2:
#        print('Debug: failed to find lane ' + str(left_fit) + ':' + str(right_fit))
        lineL.detected = False
        lineR.detected = False
    else:
        # Fit new polynomials
        left_fitx, right_fitx = fit_poly(ploty, leftx, lefty, rightx, righty)

        lineL.fit = left_fitx
        lineL.allx = leftx
        lineL.ally = lefty
        lineR.fit = right_fitx
        lineR.allx = rightx
        lineR.ally = righty
        lineL.detected = True
        lineR.detected = True

#     ## Visualization ##
#     # Create an image to draw on and an image to show the selection window
#     out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
#     window_img = np.zeros_like(out_img)
#     # Color in left and right line pixels
#     out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
#     out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

#     # Generate a polygon to illustrate the search window area
#     # And recast the x and y points into usable format for cv2.fillPoly()
#     left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
#     left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
#                               ploty])))])
#     left_line_pts = np.hstack((left_line_window1, left_line_window2))
#     right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
#     right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
#                               ploty])))])
#     right_line_pts = np.hstack((right_line_window1, right_line_window2))

#     # Draw the lane onto the warped blank image
#     cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
#     cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
#     result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
#     # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
#     ## End visualization steps ##
    
    return

## Determine the curvature of the lane and vehicle position with respect to center.

In [24]:
def measure_curvature_real(ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    global lineL, lineR
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Make sure to feed in your real data instead in your project!
    left_fit_cr = np.polyfit(ploty * ym_per_pix, lineL.fit * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, lineR.fit * xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    lineL.radius_of_curvature = left_curverad
    lineR.radius_of_curvature = right_curverad
    
    # distance in meters of vehicle center from the line
    lineL.line_base_pos = (left_fit_cr[0]*y_eval*ym_per_pix**2 + left_fit_cr[1]*y_eval*ym_per_pix + left_fit_cr[2]) - 640*xm_per_pix
    lineR.line_base_pos = (right_fit_cr[0]*y_eval*ym_per_pix**2 + right_fit_cr[1]*y_eval*ym_per_pix + right_fit_cr[2]) - 640*xm_per_pix

    # append recent coefficent
    lineL.recent_fit.append(lineL.current_fit)
    lineR.recent_fit.append(lineR.current_fit)
    # append recent position
    lineL.recent_xfitted.append(lineL.line_base_pos)
    lineR.recent_xfitted.append(lineR.line_base_pos)
    # append recent curvature
    lineL.recent_curvature.append(lineL.radius_of_curvature)
    lineR.recent_curvature.append(lineR.radius_of_curvature)

    return

## Warp the detected lane boundaries back onto the original image.

In [25]:
def warp_back(image, warped, ploty, fitL, fitR, M):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([fitL, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([fitR, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, np.linalg.inv(M), (image.shape[1], image.shape[0])) 
    
    return newwarp

## sanity check

In [26]:
def verifyLane(width_thresh=[2, 4], fit_thresh = 0.2, curb_thresh=1e4):
    valid = True
    # check if distance of left and right should be appropriate
    if (lineR.line_base_pos - lineL.line_base_pos) > width_thresh[1]:
#        print('line_base_pos large diff:' + str(lineR.line_base_pos - lineL.line_base_pos) + ' R:' + str(lineR.line_base_pos) + ' L:' + str(lineL.line_base_pos))
        valid = False
    # check if distance of left and right should be appropriate
    if (lineR.line_base_pos - lineL.line_base_pos) < width_thresh[0]:
#        print('line_base_pos small diff:' + str(lineR.line_base_pos - lineL.line_base_pos) + ' R:' + str(lineR.line_base_pos) + ' L:' + str(lineL.line_base_pos))
        valid = False
    # check if parallel
    if abs(lineL.current_fit[0] - lineR.current_fit[0]) > fit_thresh:
#        print('current_fit diff:' + str(abs(lineL.current_fit[0] - lineR.current_fit[0])))
        valid = False
    # check if curvature is similar
#     if abs(lineL.radius_of_curvature - lineR.radius_of_curvature) > curb_thresh:
#         print('curvature diff:' + str(abs(lineL.radius_of_curvature - lineR.radius_of_curvature)))
#         valid = False

    # check if curvature is similar to previous
    if lineL.n_valid > 1:
        curb_prevL = list(lineL.recent_curvature)[lineL.n_valid - 2]
        if abs(curb_prevL - lineL.radius_of_curvature) > curb_thresh:
#            print('curvature diff L:' + str(abs(curb_prevL - lineL.radius_of_curvature)) + ' pre:' + str(curb_prevL) + ' cur:' + str(lineL.radius_of_curvature))
            valid = False
    if lineR.n_valid > 1:
        curb_prevR = list(lineR.recent_curvature)[lineR.n_valid - 2]
        if abs(curb_prevR - lineR.radius_of_curvature) > curb_thresh:
#            print('curvature diff R:' + str(abs(curb_prevR - lineR.radius_of_curvature)) + ' pre:' + str(curb_prevR) + ' cur:' + str(lineR.radius_of_curvature))
            valid = False
            
    if not valid:
        if lineL.detected or lineR.detected:
            lineL.recent_fit.pop()
            lineL.recent_xfitted.pop()
            lineL.recent_curvature.pop()
            lineR.recent_fit.pop()
            lineR.recent_xfitted.pop()
            lineR.recent_curvature.pop()
            lineL.detected = False
            lineR.detected = False
    lineL.n_valid = len(lineL.recent_fit)
    lineR.n_valid = len(lineR.recent_fit)
    if lineL.n_valid > 0:
        lineL.best_curvature = np.average(lineL.recent_curvature)
        lineL.best_x = np.average(lineL.recent_xfitted)
        lineL.best_fit = np.average(np.array(lineL.recent_fit), axis=0)
    if lineR.n_valid > 0:
        lineR.best_curvature = np.average(lineR.recent_curvature)
        lineR.best_x = np.average(lineR.recent_xfitted)
        lineR.best_fit = np.average(np.array(lineR.recent_fit), axis=0)
        
    return

## pipe line

In [27]:
def advancedLaneFinding(img, mtx, dist):
    global lineL, lineR
    #import ipdb; ipdb.set_trace()
    # Apply a distortion correction to raw images
    undst = cv2.undistort(img, mtx, dist, None, mtx)

    # Use color transforms, gradients, etc., to create a thresholded binary image.
    combined = threshold_binary(undst)
    
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    warped, M = transform_perspective(combined)
    
    # Detect lane pixels and fit to find the lane boundary.
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    if lineL.detected and lineR.detected:
        search_around_poly(ploty, warped, lineL.current_fit, lineR.current_fit)
        if not lineL.detected or not lineR.detected:
            print('Warning: 1st search_around_poly falied')
            fit_polynomial(ploty, warped)
    elif lineL.n_valid > 2:
        search_around_poly(ploty, warped, lineL.best_fit, lineR.best_fit)
        if not lineL.detected or not lineR.detected:
            print('Warning: 2nd search_around_poly failed')
            fit_polynomial(ploty, warped)
    else:
        fit_polynomial(ploty, warped)

    # Determine the curvature of the lane and vehicle position with respect to center.
    measure_curvature_real(ploty)
    
    # sanity check
    verifyLane()

    # Warp the detected lane boundaries back onto the original image.
    if lineL.n_valid > 0:
        left_fitx = lineL.best_fit[0]*ploty**2 + lineL.best_fit[1]*ploty + lineL.best_fit[2]
        right_fitx = lineR.best_fit[0]*ploty**2 + lineR.best_fit[1]*ploty + lineR.best_fit[2]
        newwarp = warp_back(undst, warped, ploty, left_fitx, right_fitx, M)
    else:
        newwarp = warp_back(undst, warped, ploty, lineL.fit, lineR.fit, M)
        
    if lineL.n_valid == 0:
        print('Error: no valid fit')
    
    return undst, warped, newwarp, ploty

# keep line chracterictics
count = 0
lineL = Line(5)
lineR = Line(5)
dist_pickle = pickle.load( open( "camera_cal_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]
def process_image(img):
    global count
    count += 1
    undst, combo, newwarp, ploty = advancedLaneFinding(img, mtx, dist)
    result = cv2.addWeighted(undst, 1, newwarp, 0.3, 0)
    
    # write text
    # average of curvature
    if lineL.best_curvature is None or lineR.best_curvature is None:
        print('Error: avg_curvature None')
    else:
        avg_curvature = (lineL.best_curvature + lineR.best_curvature)//2
        cv2.putText(result, 'Radius of Curvature = ' + str(int(avg_curvature)) + '(m)', (50,80), cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(255,255,255), thickness=2)
    
    # viehcle position
    if lineL.best_x is None or lineR.best_x is None:
        print('Error: line_center None')
    else:
        line_center = (lineL.best_x + lineR.best_x)/2
        str_line_center = "{:.2f}".format(np.abs(line_center))
        if line_center > 0:
            cv2.putText(result, 'Vehicle is ' + str_line_center + 'm right of center', (50,160), cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(255,255,255), thickness=2)
        elif line_center < 0:
            cv2.putText(result, 'Vehicle is ' + str_line_center + 'm left of center', (50,160), cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(255,255,255), thickness=2)
        else:
            cv2.putText(result, 'Vehicle is at center', (50,160), cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(255,255,255), thickness=2)

    return result

## video

In [28]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [30]:
white_output = 'project_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("project_video.mp4").subclip(18, 26)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False, codec='mpeg4')

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4



 12%|█▏        | 152/1261 [01:25<10:26,  1.77it/s]


 24%|██▍       | 303/1261 [02:50<08:59,  1.78it/s]


 36%|███▌      | 454/1261 [04:15<07:34,  1.78it/s]


 48%|████▊     | 605/1261 [05:40<06:08,  1.78it/s]


...

 72%|███████▏  | 907/1261 [08:31<03:19,  1.77it/s]


 84%|████████▍ | 1057/1261 [09:56<01:55,  1.77it/s]


 96%|█████████▌| 1205/1261 [11:21<00:31,  1.77it/s]


100%|█████████▉| 1260/1261 [11:52<00:00,  1.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

CPU times: user 36min 2s, sys: 2min 52s, total: 38min 55s
Wall time: 11min 52s


In [31]:
white_output = 'challenge_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False, codec='mpeg4')

[MoviePy] >>>> Building video challenge_video_out.mp4
[MoviePy] Writing video challenge_video_out.mp4



 32%|███▏      | 155/485 [01:28<03:08,  1.75it/s]


 64%|██████▎   | 309/485 [02:55<01:39,  1.76it/s]


 95%|█████████▌| 463/485 [04:21<00:12,  1.77it/s]


100%|██████████| 485/485 [04:33<00:00,  1.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_out.mp4 

CPU times: user 13min 42s, sys: 1min 12s, total: 14min 55s
Wall time: 4min 33s


In [32]:
white_output = 'harder_challenge_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False, codec='mpeg4')

[MoviePy] >>>> Building video harder_challenge_video_out.mp4
[MoviePy] Writing video harder_challenge_video_out.mp4



 13%|█▎        | 152/1200 [01:31<10:29,  1.66it/s]


 25%|██▌       | 303/1200 [03:02<08:59,  1.66it/s]


 38%|███▊      | 454/1200 [04:33<07:29,  1.66it/s]


 50%|█████     | 605/1200 [06:00<05:54,  1.68it/s]


 63%|██████▎   | 756/1200 [07:30<04:24,  1.68it/s]


 76%|███████▌  | 907/1200 [09:00<02:54,  1.68it/s]


 88%|████████▊ | 1057/1200 [10:24<01:24,  1.69it/s]


100%|█████████▉| 1199/1200 [11:48<00:00,  1.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_out.mp4 

CPU times: user 36min 25s, sys: 2min 51s, total: 39min 16s
Wall time: 11min 48s
